In [2]:
#instalacion pip de dependencias necesarias
!pip install pyodbc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.0/336.0 kB 20.1 MB/s eta 0:00:00


In [ ]:
#carga de datos desde MSSQL y guardado en Storage
import pyodbc
from google.cloud import storage
import csv

# Configuración de conexión MSSQL
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=192.168.1.41;'
    'DATABASE=InsightHub;'
    'UID=grupo2;'
    'PWD=henry'
)
cursor = conn.cursor()

# Consulta y exportación a CSV
query = "SELECT * FROM mi_tabla"
cursor.execute(query)

csv_file = "/home/gaz/Descargas/InsightHub.csv"
with open(csv_file, "w", newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow([col[0] for col in cursor.description])  # Nombres de columnas
    writer.writerows(cursor.fetchall())

# Subir a Cloud Storage
client = storage.Client()
bucket = client.get_bucket("insight_hub")
blob = bucket.blob("InsightHub")
blob.upload_from_filename(csv_file)

print("Archivo subido a Cloud Storage: datos.csv")


In [ ]:
#carga de datos a BigQuery
from google.cloud import bigquery

# Configuración de cliente BigQuery
client = bigquery.Client()

# Configuración de la tabla de destino
table_id = "starry-aegis-443515-b9.insight_hub.dbo_Compras"
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
)

# Cargar desde Cloud Storage
uri = "gs://insight_hub/InsightHub.csv"
load_job = client.load_table_from_uri(uri, table_id, job_config)

# Esperar la finalización de la carga
load_job.result()
print(f"Datos cargados a {table_id}")


La transformacion de datos se hace a través de queries SQL en BigQuery

In [ ]:
#Automatizacion de carga
import google.cloud.bigquery as bigquery

def cargar_a_bigquery(data, context):
    # Configuración de cliente BigQuery
    client = bigquery.Client()

    # Configuración de la tabla de destino
    table_id = "starry-aegis-443515-b9.insight_hub.dbo_Compras"
    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1,
        autodetect=True,
    )

    # URI del archivo que activó la función
    bucket_name = data["bucket"]
    file_name = data["name"]
    uri = f"gs://{bucket_name}/{file_name}"

    # Cargar archivo a BigQuery
    load_job = client.load_table_from_uri(uri, table_id, job_config)
    load_job.result()
    print(f"Archivo {file_name} cargado a {table_id}")
